In [2]:
! pip install pandas

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [3]:
! pip install torch 

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [4]:
! pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip available: 22.2.1 -> 22.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import pandas as pd 
import numpy as np

#Please, import the dataset from 
## https://uvaauas.figshare.com/articles/dataset/Norman_descriptors_and_predicted_LC50/20089787
#Since it is too big I cannot include it into the zip folder


df = pd.read_csv('LC50 dataset.csv')
smiles_list = np.asanyarray(df.SMILES)
df.rename(columns = {'LC50[-LOG(mol/L)]':'LC50'}, inplace = True)
LC50_list = np.asanyarray(df.LC50)

all_categories = ['<3', '3-5', '>6']
filtered = []

for lower, upper in [(0, 4), (4, 6), (6, 9)]:
    curr = []
    for idx in range(len(LC50_list)):
        if LC50_list[idx] >= lower and LC50_list[idx] < upper:
            curr.append(smiles_list[idx])      
    filtered.append(curr)

from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker

# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Build the category_lines dictionary, a list of lines per category
category_lines = {}
category_lines[all_categories[0]] = [unicodeToAscii(line) for line in filtered[0]]
category_lines[all_categories[1]] = [unicodeToAscii(line) for line in filtered[1]]
category_lines[all_categories[2]] = [unicodeToAscii(line) for line in filtered[2]]

n_categories = len(all_categories)

In [6]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [7]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

In [8]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item() / input_line_tensor.size(0)

In [ ]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

rnn = RNN(n_letters, 128, n_letters)

n_iters = 100000
print_every = 5000
plot_every = 500
all_metrics = []
total_metric = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, metric = train(*randomTrainingExample())
    total_metric += metric

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, metric))

    if iter % plot_every == 0:
        all_metrics.append(total_metric / plot_every)
        total_metric = 0

2m 10s (5000 5%) 1.8853
3m 58s (10000 10%) 1.2975
5m 31s (15000 15%) 0.9424
6m 54s (20000 20%) 1.6065
7m 58s (25000 25%) 1.7470
8m 54s (30000 30%) 2.1759


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.xlabel('Iteration')
plt.ylabel('Log Loss')
plt.plot(all_metrics);

In [10]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='C'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('<3', 'CN')

COCCCCCCCCCCOCCCCCCCC
NCCCCCCCCCCCCCCCCCCCC
